## Import and cleaning

In [53]:
import pandas as pd
import json
import pyomo.environ as pyo

with open("../data/network_graph_2024_06_12.json") as f:
    d = json.load(f)

nodes = d["nodes"]
channels = d["edges"]

nodes = pd.DataFrame(nodes)
channels = pd.DataFrame(channels)

## Change data types
channels["capacity"] = channels["capacity"].astype(int)
channels["last_update"] = pd.to_datetime(channels["last_update"], unit = 's')

## Filter out channels that are unused (no update time)
channels = channels[channels["last_update"] > "1970-01-01"]

## Filter out channels with nodes with no policy registered for this describegraph query (aka not reachable now)
channels = channels[pd.notnull(channels["node1_policy"]) & pd.notnull(channels["node2_policy"])]

channels = channels.filter(items=['channel_id',
                       'node1_pub',
                       'node2_pub',
                       'capacity',
                       'node1_policy',
                       'node2_policy'
                       ])



From these data we only need information that are strictly related to path finding for channels, thus we only need:

- channel peers
- channel id
- capacity
- nodes policy:
    - fee base msat
    - fee rate milli msat

In [54]:
channels.iloc[0]

channel_id                                     627185621808578560
node1_pub       027d7f94667974b10d3e8330de403111229669273dc902...
node2_pub       03abf6f44c355dec0d5aa155bdbdd6e0c8fefe318eff40...
capacity                                                  3000000
node1_policy    {'time_lock_delta': 144, 'min_htlc': '1000', '...
node2_policy    {'time_lock_delta': 30, 'min_htlc': '1000', 'f...
Name: 4650, dtype: object

In [55]:
channels['node1_fee_base_msat'] = channels['node1_policy'].apply(lambda x: x['fee_base_msat'])
channels['node2_fee_base_msat'] = channels['node2_policy'].apply(lambda x: x['fee_base_msat'])

channels['node1_fee_rate_milli_msat'] = channels['node1_policy'].apply(lambda x: x['fee_rate_milli_msat'])
channels['node2_fee_rate_milli_msat'] = channels['node2_policy'].apply(lambda x: x['fee_rate_milli_msat'])

channels = channels.filter(items=['channel_id',
                                  'node1_pub',
                                  'node2_pub',
                                  'capacity',
                                  'node1_fee_base_msat',
                                  'node1_fee_rate_milli_msat',
                                  'node2_fee_base_msat',
                                  'node2_fee_rate_milli_msat'
                                  ])

In [56]:
channels[0:5]

,channel_id,node1_pub,node2_pub,capacity,node1_fee_base_msat,node1_fee_rate_milli_msat,node2_fee_base_msat,node2_fee_rate_milli_msat
4650,627185621808578560,027d7f94667974b10d3e8330de403111229669273dc902...,03abf6f44c355dec0d5aa155bdbdd6e0c8fefe318eff40...,3000000,250,100,1000,1700
5447,674672429454655488,027d7f94667974b10d3e8330de403111229669273dc902...,02ca4c46faa50b2d961f4ad2e9d00c90826077b6a282b7...,500000,400,1000,999,999
5949,700826512665280513,03a858dea8b0ece46ac5543dd89611fe8e17706bbc4727...,03abf6f44c355dec0d5aa155bdbdd6e0c8fefe318eff40...,55000,1,1,1000,1700
6081,713331258365837313,03864ef025fde8fb587d989186ce6a4a186895ee44a926...,03a6680e79e30f050d4f32f1fb9d046cc6efb5ed4cc99e...,1100000,1000,499,0,100000
6140,717789778064113665,030e3c5473d6fd21844958bf4c75f9ebbd751f7f3f9302...,03a6680e79e30f050d4f32f1fb9d046cc6efb5ed4cc99e...,500000,1000,1,30000,100
